In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import pandas as pd
import requests
import os
import re
import glob
from fastai import *
from fastai.vision import *
import matplotlib.pyplot as plt
import torch
import imghdr

In [3]:
path='/content/drive/My Drive/Colab Notebooks/SSS_AI_assignment/'
path = os.path.dirname(path)
test_path='/content/drive/My Drive/Colab Notebooks/SSS_AI_assignment/test/'
img_path='/content/drive/My Drive/Colab Notebooks/SSS_AI_assignment/image/'

In [4]:
print ('Available devices ', torch.cuda.device_count())
print ('Current cuda device ', torch.cuda.current_device())
torch.cuda.set_device(0)

Available devices  1
Current cuda device  0


In [5]:
with open(os.path.join(path,'assignment_test_dataset.csv')) as fraw:
    raw=pd.read_csv(fraw)
raw.columns=["","id","sku","name","url_key","url_path","type","book_cover","short_description","price","list_price","price_usd","badges","discount","discount_rate","rating_average","review_count","order_count","favourite_count","thumbnail_url","has_ebook","inventory_status","is_visible","productset_group_name","is_fresh","seller","is_flower","is_gift_card","inventory","url_attendant_input_form","master_id","salable_type","data_version","categories","meta_title","meta_description","meta_keywords","liked","rating_summary","description","return_policy","warranty_policy","brand","seller_specifications","current_seller","other_sellers","configurable_options","configurable_products","specifications","product_links","services_and_promotions","promotions","stock_item","installment_info","video_url","youtube","is_seller_in_chat_whitelist","last_update","extra_feature"]
url=raw['thumbnail_url']
id=raw['id']

In [6]:
# for index,item in enumerate(url):
#     img_data = requests.get(item).content
#     with open(os.path.join(test_path,('img_'+f'{id[index]}'+'.jpg')), 'wb') as handler:
#         handler.write(img_data)

In [7]:
list_dir=os.listdir(test_path)

In [8]:
list_df=[]
for ele_list_dir in list_dir:
  list_ele_df=[]
  img_name = str(ele_list_dir)
  list_ele_df.append(str(img_name))
  list_df.append(list_ele_df)
test_df=pd.DataFrame(list_df,columns=['image_name'])
# test_df.to_csv(os.path.join(test_path,'test_set.csv'),sep=",", header=True, index=False, mode='w')

In [9]:
data = ImageDataBunch.from_csv(img_path, csv_labels='train_labels.csv', ds_tfms=get_transforms(), size=150)
data.normalize(imagenet_stats)

ImageDataBunch;

Train: LabelList (4072 items)
x: ImageList
Image (3, 150, 150),Image (3, 150, 150),Image (3, 150, 150),Image (3, 150, 150),Image (3, 150, 150)
y: CategoryList
ao-len-cardigan,ao-len-cardigan,ao-len-cardigan,ao-len-cardigan,ao-len-cardigan
Path: /content/drive/My Drive/Colab Notebooks/SSS_AI_assignment/image;

Valid: LabelList (1018 items)
x: ImageList
Image (3, 150, 150),Image (3, 150, 150),Image (3, 150, 150),Image (3, 150, 150),Image (3, 150, 150)
y: CategoryList
ao-dai,ao-nguc,ao-dai,dam-nu,ao-len-cardigan
Path: /content/drive/My Drive/Colab Notebooks/SSS_AI_assignment/image;

Test: None

In [10]:
learn = create_cnn(data, models.resnet34, metrics=accuracy)

/usr/local/lib/python3.6/dist-packages/fastai/vision/learner.py:109: UserWarning: `create_cnn` is deprecated and is now named `cnn_learner`.
  warn("`create_cnn` is deprecated and is now named `cnn_learner`.")
Downloading: "https://download.pytorch.org/models/resnet34-333f7ec4.pth" to /root/.cache/torch/hub/checkpoints/resnet34-333f7ec4.pth


In [11]:
learn.load(os.path.join(path,'stage-2_arch-34_sz-150'))

Learner(data=ImageDataBunch;

Train: LabelList (4072 items)
x: ImageList
Image (3, 150, 150),Image (3, 150, 150),Image (3, 150, 150),Image (3, 150, 150),Image (3, 150, 150)
y: CategoryList
ao-len-cardigan,ao-len-cardigan,ao-len-cardigan,ao-len-cardigan,ao-len-cardigan
Path: /content/drive/My Drive/Colab Notebooks/SSS_AI_assignment/image;

Valid: LabelList (1018 items)
x: ImageList
Image (3, 150, 150),Image (3, 150, 150),Image (3, 150, 150),Image (3, 150, 150),Image (3, 150, 150)
y: CategoryList
ao-dai,ao-nguc,ao-dai,dam-nu,ao-len-cardigan
Path: /content/drive/My Drive/Colab Notebooks/SSS_AI_assignment/image;

Test: None, model=Sequential(
  (0): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): BasicBlock(
   

In [12]:
def choose_label(out_pred):
  out_clas=[]
  tens_values, tens_indi = out_pred.topk(3)
  list_values = tens_values.tolist()
  list_indi = tens_indi.tolist()
  if max (list_values) > 0.8:
    # determine top 1
    index = list_values.index(max(list_values))
    label = learn.data.classes[list_indi[index]]
    out_clas.append(label)
  else:
    for ele_clas in list_indi:
      label = learn.data.classes[ele_clas]
      out_clas.append(label)
  return out_clas

In [18]:
list_dir=os.listdir(test_path)
list_out=[]
except_list=['test_set.csv', 'test_clasify.csv', 'img_13900063.jpg', 'img_31205836.jpg', 'img_24492126.jpg']
for ele_img_name in list_dir:
  list_test=[]
  if ele_img_name not in except_list:
    str_ele = str(ele_img_name)
    str_ele = str_ele.replace(r"'",r'').replace(r"[",r'').replace(r"]",r'')

    id_img= str_ele.replace(r'img_','').replace(r'.jpg','')
    list_test.append(id_img)
    # try:
    img = open_image(os.path.join(test_path,f'{str_ele}'))
    # except:
      # break
    pred_class, pred_idxs, outputs = learn.predict(img)
    
    list_categ=choose_label(outputs)
    label = ','.join(list_categ)
    list_test.append(label)
    list_out.append(list_test)


In [20]:
test_clasify=pd.DataFrame(list_out,columns=['id','cloth_label'])
test_clasify.to_csv(os.path.join(test_path,'test_clasify.csv'),sep=",", header=True, index=False, mode='w')
# print(len(list_out))

1502


In [15]:
error_list=[]
for ele_img_name in list_dir:
  str_ele = str(ele_img_name)
  str_ele = str_ele.replace(r"'",r'').replace(r"[",r'').replace(r"]",r'')
  try:
    img = open_image(os.path.join(path,f'{str_ele}'))
  except:
    error_list.append(str_ele)
print(error_list)